In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from gears import PertData, GEARS
import numpy as np
import pandas as pd
from tqdm import tqdm
import scanpy as sc
import anndata
from scipy.sparse import csr_matrix

In [3]:
data_dir = '/mlbio_scratch/vinas/sc_perturbation_benchmark/data/replogle_k562_gwps_2022'

#### Download data

Data access: https://gwps.wi.mit.edu/

* Files:
  * K562_essential_raw_singlecell.h5ad (https://plus.figshare.com/articles/dataset/_Mapping_information-rich_genotype-phenotype_landscapes_with_genome-scale_Perturb-seq_Replogle_et_al_2022_processed_Perturb-seq_datasets/20029387)
  * annotated_embedding_coordinates.csv (https://plus.figshare.com/articles/dataset/_Mapping_information-rich_genotype-phenotype_landscapes_with_genome-scale_Perturb-seq_Replogle_et_al_2022_-_commonly_requested_supplemental_files/21632564/1)

In [4]:
adata = anndata.read_h5ad(f'{data_dir}/K562_gwps_raw_singlecell_01.h5ad')
df_sub = pd.read_csv(f'{data_dir}/annotated_embedding_coordinates.csv')

#### scGPT processing
1. Select genes in annotated embedding coordinates, corresponding to perturbations that elicit strong phenotype changes.
2. Downsample 100 samples per perturbation and 2500 control samples
3. Normalize data (unsure why the scGPT authors performed HVG selection after downsampling).
4. Keep HVGs and genes that were perturbed

In [5]:
adata_ = adata[adata.obs['gene'].isin(list(df_sub.gene.values)+['non-targeting']), :]

In [8]:
# Subsample to 100 samples per perturbation and 2500 control samples
target_cells = 100
cluster_key = 'gene'
adatas = [adata_[adata_.obs[cluster_key]==clust] for clust in adata_.obs[cluster_key].cat.categories]
for dat in tqdm(adatas):
    if dat.n_obs > target_cells:
        if dat.obs[cluster_key].cat.categories.values[0] != 'non-targeting':
            sc.pp.subsample(dat, n_obs=target_cells)
        else:
            sc.pp.subsample(dat, n_obs=2500)
adata_downsampled_ = adatas[0].concatenate(*adatas[1:])

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1970/1970 [00:09<00:00, 201.08it/s]


In [9]:
sc.pp.normalize_total(adata_downsampled_)
sc.pp.log1p(adata_downsampled_)
sc.pp.highly_variable_genes(adata_downsampled_, n_top_genes=5000, subset=False)
hvg_flag_ = adata_downsampled_.var['highly_variable'].values
gene_flag_ = adata_downsampled_.var['gene_name'].isin(adata_downsampled_.obs['gene'].values).values
select_flag_ = np.logical_or(hvg_flag_, gene_flag_)
condition_flag_ = adata_downsampled_.obs['gene'].isin(adata_downsampled_.var['gene_name'].values.tolist()+['non-targeting']).values
adata_subset = adata_downsampled_[condition_flag_, select_flag_]

In [ ]:
# Prepare data in GEARS format
adata_subset.obs['condition'] = [i+'+ctrl' for i in adata_subset.obs['gene'].values]
adata_subset.obs['condition'] = adata_subset.obs['condition'].replace({'non-targeting+ctrl': 'ctrl'})
adata_subset.obs['cell_type'] = 'K562'

In [11]:
out_dir = data_dir
pert_data = PertData(out_dir)
adata_subset.X = csr_matrix(adata_subset.X)
pert_data.new_data_process(dataset_name = 'k562_1900_{}'.format(target_cells), adata = adata_subset)

Downloading...
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9.46M/9.46M [00:00<00:00, 9.85MiB/s]
Downloading...
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 559k/559k [00:00<00:00, 1.42MiB/s]
Creating pyg object for each cell in the data...
Creating dataset file...
  1%|█▍                                                                                                                                                                                                                                                                        | 10/1823 [00

AC118549.1+ctrl


  7%|██████████████████▌                                                                                                                                                                                                                                                      | 128/1823 [01:07<13:37,  2.07it/s]

C7orf26+ctrl


  7%|███████████████████▏                                                                                                                                                                                                                                                     | 132/1823 [01:09<14:29,  1.94it/s]

...

 77%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                                                           | 1410/1823 [12:33<02:51,  2.41it/s]

SEM1+ctrl


 90%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                         | 1646/1823 [14:39<01:51,  1.59it/s]

TRABD+ctrl


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1823/1823 [16:21<00:00,  1.86it/s]
Done!
Saving new dataset pyg object at /mlbio_scratch/vinas/sc_perturbation_benchmark/data/replogle_k562_gwps_2022/k562_1900_100/data_pyg/cell_graphs.pkl
Done!


In [ ]:
# Store data in sparse format
adata.X = csr_matrix(adata.X)
adata.write_h5ad(f'{data_dir}/K562_gwps_raw_singlecell_01.h5ad')